In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('app1').getOrCreate()
df = spark.read.csv("customer.csv",header=True)
df.show()

+-----------+-------------+---------+-------------------+----------------+-------+
|Customer_No|    Card_type|     Date|           Category|Transaction Type| Amount|
+-----------+-------------+---------+-------------------+----------------+-------+
|    1000501|Platinum Card| 1/1/2018|           Shopping|           debit|  11.11|
|    1000501|     Checking| 1/2/2018|    Mortgage & Rent|           debit|1247.44|
|    1000501|  Silver Card| 1/2/2018|        Restaurants|           debit|  24.22|
|    1000501|Platinum Card| 1/3/2018|Credit Card Payment|          credit|2298.09|
|    1000501|Platinum Card| 1/4/2018|      Movies & DVDs|           debit|  11.76|
|    1000501|  Silver Card| 1/5/2018|        Restaurants|           debit|  25.85|
|    1000501|  Silver Card| 1/6/2018|   Home Improvement|           debit|  18.45|
|    1000501|     Checking| 1/8/2018|          Utilities|           debit|     45|
|    1000501|  Silver Card| 1/8/2018|   Home Improvement|           debit|  15.38|
|   

In [2]:
df.count()

105

In [11]:
df.rdd.getNumPartitions()

1

In [12]:
df.select("Customer_No").distinct().show()

+-----------+
|Customer_No|
+-----------+
|    1001863|
|    1001368|
|    1000210|
|    1000531|
|    1000654|
|    1002324|
|    1000501|
+-----------+



In [16]:
df.repartition(7,"Customer_No").rdd.getNumPartitions()

7

In [26]:
from pyspark.sql.functions import spark_partition_id

df1 = df.repartition(7,"Customer_No").withColumn("partitionId",spark_partition_id()).groupBy("partitionId").count()
df1.show()

+-----------+-----+
|partitionId|count|
+-----------+-----+
|          2|   31|
|          3|   13|
|          4|   33|
|          6|   28|
+-----------+-----+



In [22]:
df1 = df.repartition(7, "Customer_No")
df1 = df1.withColumn("partitionId", spark_partition_id())
rdd = df1.rdd
rdd = rdd.mapPartitions(lambda iter: [len(list(iter))])

partition_counts = rdd.collect()
for partition_id, count in enumerate(partition_counts):
    print(f"Partition {partition_id}: {count}")

Partition 0: 0
Partition 1: 0
Partition 2: 31
Partition 3: 13
Partition 4: 33
Partition 5: 0
Partition 6: 28


In [23]:
df.write.partitionBy("Customer_No")

In [24]:
df.repartition("Customer_No").write.format('csv').option("header",True).mode("overwrite").save("demo_repartition_file_01")

In [25]:
df.write.format('csv').partitionBy("Customer_No").option("header",True).mode("overwrite").save("demo_partitionBy_file_01")